In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten, Lambda, Conv2D, AveragePooling2D, BatchNormalization
from keras.layers import merge, Convolution2D
from keras import Input, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
from model_keras import StochasticDepth
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '3'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
nb_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train = np.transpose(x_train.astype('float32'), (0, 1, 2, 2))
# x_test = np.transpose(x_test.astype('float32'), (0, 3, 1,2))
# reorder dimensions for tensorflow
mean = np.mean(x_train, axis=0, keepdims=True)
std = np.std(x_train)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

model = StochasticDepth().model


W0917 01:48:53.089184 140290381059840 deprecation_wrapper.py:119] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 01:48:53.091479 140290381059840 deprecation_wrapper.py:119] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 01:48:53.097542 140290381059840 deprecation_wrapper.py:119] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0917 01:48:53.126787 140290381059840 deprecation_wrapper.py:119] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is dep

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


W0917 01:48:53.990919 140290381059840 deprecation_wrapper.py:119] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0917 01:48:54.319988 140290381059840 deprecation.py:323] From /home/kim3/anaconda3/envs/fedml/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4160: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


NameError: name 'AveragePooling2D' is not defined

In [ ]:
sgd = SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True)

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 128
nb_epoch = 200
data_augmentation = True

# Learning rate schedule
def lr_sch(epoch):
    if epoch < nb_epoch * 0.5:
        return 0.1
    elif epoch < nb_epoch * 0.75:
        return 0.01
    else:
        return 0.001

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_sch)

In [ ]:
datagen_train = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0.125,
        height_shift_range=0.125,
        horizontal_flip=True,
        vertical_flip=False)
datagen_train.fit(x_train)

# fit the model on the batches generated by datagen.flow()
history = model.fit_generator(datagen_train.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
                              samples_per_epoch=x_train.shape[0], 
                              nb_epoch=nb_epoch,
                              validation_data=(x_test, y_test),
                              callbacks=[lr_scheduler])